In [2]:
import cv2
import numpy as np
import math

In [11]:
#webcam capture
capture=cv2.VideoCapture(0)

while capture.isOpened():
    #capture frames from camera
    ret, frame=capture.read()
    
    #draw a rectangle over the frame..hands only insside this rectangle will be identified
    #rectangle(image, (x,y coordinates of top left corner), (x,y coord of bottom right corner) , color, thickness )
    cv2.rectangle(frame, (100,100) , (300, 300) , (0,255,0) , 0)  #green rect
    
    #capture rectnagular portion from frame
    crop_image= frame[100:300 , 100:300]
    
    #apply Gaussian blur--blurring is done to smoothen image--3,3 is the blur kernel
    blur=cv2.GaussianBlur(crop_image, (3,3) ,0)
    
    #change color space from BGR-->HSV
    hsv=cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    
    # performl Thresholding--Create a binary image where white will be skin colors and rest is black
    #check: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_colorspaces/py_colorspaces.html
    #you can say that for filtering skin color we have color range [2, 0, 0] to [20, 255, 255]--atleast I could not figure it out
    mask2 = cv2.inRange(hsv, np.array([2, 0, 0]), np.array([20, 255, 255]))
    
    #apply morphological functions--dilation and erosion. 
    #dilation will add some pixels that may have been removed
    #erosion will remove some pixels which may have been added
    kernel=np.ones((5,5)) #kernel for morphological operations
    dilation=cv2.dilate(mask2, kernel, iterations=1)
    erosion=cv2.dilate(dilation, kernel, iterations=1)
    
    #after morphological operations, we get new images, on which we need to apply Gaussian blur for smoothening
    filtered=cv2.GaussianBlur(erosion, (3,3) ,0)
    
    #perform thresholding--this means convert the image to B&W format.
    #the last parameter THRESH_BINARY means that we want background in black and foreground in white
    ret, thresh = cv2.threshold(filtered, 127, 255, cv2.THRESH_BINARY)
    
    #find contour
    #Contours are defined as the line joining all the points along the boundary of an image that are having the same intensity
    #findContour() function helps in extracting the contours from the image. 
    #It works best on binary images, so we should first apply thresholding techniques
    contours,_= cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    try:
        # there would be multiple contours. Find contour with maximum area from contours
        #max(contours, key=someFunc())
        contour = max(contours, key=lambda x: cv2.contourArea(x))
        
        #cv2.boundingRect() function of OpenCV is used to draw an approximate rectangle around the binary image. 
        #This function is used mainly to highlight the region of interest after obtaining contours from an image.
        x, y, w, h = cv2.boundingRect(contour)   #x, y--coord of top left corner, w,h--width, height of rectangle
        
        #draw this rectangle over image
        cv2.rectangle(crop_image, (x, y), (x + w, y + h), (0, 0, 255), 1)
        
        # Find convex hull
        hull = cv2.convexHull(contour)

        # Draw contour
        drawing = np.zeros(crop_image.shape, np.uint8)  #get a black background
        cv2.drawContours(drawing, [contour], -1, (0, 255, 0), 0) #green
        cv2.drawContours(drawing, [hull], -1, (0, 0, 255), 0)    #red
        
        #finding convexity defects
        defects = cv2.convexityDefects(contour, hull)
        
    except:
        pass
        
        
        
        
        
        
        
        
    
    # Show required images
    cv2.imshow("Gesture", frame)
    all_image = np.hstack((drawing, crop_image))
    cv2.imshow('Contours', all_image)

    # Close the camera if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break
    
capture.release()
cv2.destroyAllWindows()    